# Plan of Action

In [16]:
import matplotlib.pyplot as plt
import os
import importlib
import pandas

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
from pathlib import Path

# Add parent directory to sys.path
parent_dir = Path("..").resolve()
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))
import itertools
from sklearn.metrics import confusion_matrix
import common_utils
importlib.reload(common_utils)
from common_utils import *

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

np.random.seed(69)
tf.random.set_seed(69)

## Import Dataset

In [8]:
# normal data
image_paths_csv = pd.read_csv("./processed_data/image_paths_7_classes.csv")
#aug data
image_paths_csv_aug = pd.read_csv("./processed_data/image_paths_7_classes_aug.csv")



In [9]:
image_paths_csv

,path,age,age_bin,age_bin_label
0,../raw_data2/face_age/001/1000.png,1,0,Infant/Toddler (1–4)
1,../raw_data2/face_age/001/1018.png,1,0,Infant/Toddler (1–4)
2,../raw_data2/face_age/001/1030.png,1,0,Infant/Toddler (1–4)
3,../raw_data2/face_age/001/1062.png,1,0,Infant/Toddler (1–4)
4,../raw_data2/face_age/001/1068.png,1,0,Infant/Toddler (1–4)
...,...,...,...,...
9567,../raw_data2/face_age/100/2694.png,100,6,Seniors (75+)
9568,../raw_data2/face_age/100/6408.png,100,6,Seniors (75+)
9569,../raw_data2/face_age/101/1554.png,101,6,Seniors (75+)
9570,../raw_data2/face_age/110/3906.png,110,6,Seniors (75+)


## Organize Dataset

In [10]:

data= load_tensors_from_paths_csv(paths_csv=image_paths_csv)
aug_data=load_tensors_from_paths_csv(paths_csv=image_paths_csv_aug)


# Image Greyscale Function

# CNN Architecture

Batch 1: config_1
A bunch of options based on the suggestions in the assignment.
Best models: \
    {"name": "B2_deep", "conv_filters": [32, 64, 128, 256, 512]}, \ 
    {"name": "K1_k7_deep", "kernel_size": 7, "conv_filters": [64, 128, 256]} \
    {"name": "E4_epochs_300", "epochs": 300}, \
    {"name": "J2_dense_256_128", "dense_units": [256, 128]}, acc: 0.4096 \
    {"name": "A3_relu_k5", "activation": "relu", "kernel_size": 5}, \
Worst model: \
    {"name": "I1_deep_selu", "activation": "selu", "conv_filters": [64, 128, 256, 512]}, \
    {"name": "I2_shallow_tanh", "activation": "tanh", "conv_filters": [16, 32]}, \
    {"name": "A4_selu_k7", "activation": "selu", "kernel_size": 7}, \
    {"name": "A2_tanh_k3", "activation": "tanh", "kernel_size": 3}, \
    {"name": "B1_shallow", "conv_filters": [32, 64]}, \
    
Batch 2: config_2 \
Tried fixing the bad, improving the best \
Best models: \
{"name": "A3B_relu_k5_improved", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256], "dropout_rate": 0.5}, \
{"name": "B2B_deep_bn", "conv_filters": [32, 64, 128, 256, 512], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True}, \
{"name": "A4B_relu_k7", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128]},

Bad models: \
{"name": "I2B_mid_relu", "activation": "relu", "conv_filters": [32, 64, 128]}, \
{"name": "J2B_dense_256_128", "dense_units": [256, 128], "activation": "relu", "dropout_rate": 0.5, 
 "batch_norm_dense": True}, acc: 0.3898 \
{"name": "E4B_epochs_300", "epochs": 300, "dropout_rate": 0.5, "lr_schedule": "reduce_on_plateau"}, didn't improve much  \



In [11]:
# first batch, a bunch of random-ish models
config_1 = [
    {"name": "baseline"},
    
    {"name": "A1_relu_k3", "activation": "relu", "kernel_size": 3},
    {"name": "A3_relu_k5", "activation": "relu", "kernel_size": 5},
    
    {"name": "B2_deep", "conv_filters": [32, 64, 128, 256, 512]},
    
    {"name": "J1_dense_512", "dense_units": [512]},
    {"name": "J2_dense_256_128", "dense_units": [256, 128]},
    
    {"name": "E1_dense1", "dense_units": [128]},
    {"name": "E2_dense3", "dense_units": [256, 128, 64]},
    
      # Grayscale (1-channel) input
    {"name": "F1_gray", "channels": 1},
    {"name": "F2_gray_k5", "channels": 1, "kernel_size": 5},
    {"name": "F3_gray_deep", "channels": 1, "conv_filters": [32, 64, 128, 256]},
    {"name": "F4_gray_dense2", "channels": 1, "dense_units": [256, 128]},

    
    {"name": "H1_dropout_0.3", "dropout_rate": 0.3},
    {"name": "H2_dropout_0.7", "dropout_rate": 0.7},
    
        # Batch size variations
    {"name": "B3_batch_128", "batch_size": 128},
    {"name": "B4_batch_64", "batch_size": 64},
    
        # Epoch variations
    {"name": "E1_epochs_60", "epochs": 60},
    {"name": "E2_epochs_180", "epochs": 180},
    {"name": "E3_epochs_240", "epochs": 240},
    {"name": "E4_epochs_300", "epochs": 300},

    {"name": "K1_k7_deep", "kernel_size": 7, "conv_filters": [64, 128, 256]}
    ]

# second batch, improve on best, fix worse
config_2 = [
    {"name": "baseline"},
    # Improved version of "B2_deep"
    {"name": "B2B_deep_bn", "conv_filters": [32, 64, 128, 256, 512], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True},
    # Improved version of "K1_k7_deep"
    {"name": "K1B_k7_deep_bn", "kernel_size": 7, "conv_filters": [64, 128, 256, 512], "activation": "relu", "dropout_rate": 0.4, "batch_norm": True},
    # Improved version of "E4_epochs_300"
    {"name": "E4B_epochs_300", "epochs": 300, "dropout_rate": 0.5, "lr_schedule": "reduce_on_plateau"},
    # Improved version of "J2_dense_256_128"
    {"name": "J2B_dense_256_128", "dense_units": [256, 128], "activation": "relu", "dropout_rate": 0.5, "batch_norm_dense": True},
    # Improved version of "A3_relu_k5"
    {"name": "A3B_relu_k5_improved", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256], "dropout_rate": 0.5},
    # Fix for "I1_deep_selu"
    {"name": "I1B_deep_relu_bn", "activation": "relu", "conv_filters": [64, 128, 256, 512], "dropout_rate": 0.4},


    # Fix for "I2_shallow_tanh"
    {"name": "I2B_mid_relu", "activation": "relu", "conv_filters": [32, 64, 128]},
    # Fix for "A4_selu_k7"
    {"name": "A4B_relu_k7", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128]},
    # Fix for "A2_tanh_k3"
    {"name": "A2B_relu_k3_deeper", "activation": "relu", "kernel_size": 3, "conv_filters": [32, 64, 128, 256]},
    # Fix for "B1_shallow"
    {"name": "B1B_deep_dense", "conv_filters": [32, 64, 128], "dense_units": [256, 128]}
]

# third batch, take the best from batch 2 and try on 7 classes instead of 13.
config_3 = [
    {"name": "baseline"},
    {"name": "A3B_relu_k5_improved_7_classes", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256], "dropout_rate": 0.5}, 
    {"name": "B2B_deep_bn_7_classes", "conv_filters": [32, 64, 128, 256, 512], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True}, 
    {"name": "A4B_relu_k7_7_classes", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128]},
]

# forth batch, will try the same 7 classes, but without normalising the images
# This just made it worse. I have also decided to drop the epochs to 60 since validation is not increasing after that
config_4 = [
    {"name": "baseline_7_classes_non_normalized"},
    {"name": "A3B_relu_k5_improved_7_classes_non_normalized", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256], "dropout_rate": 0.5, "normalize": False}, 
    {"name": "B2B_deep_bn_7_classes_non_normalized", "conv_filters": [32, 64, 128, 256, 512], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"normalize": False}, 
    {"name": "A4B_relu_k7_7_classes_non_normalized", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128],"normalize": False},
]

# fifth batch, going back to normalized photos, but will test the same models with matched dense layers and more of them  version of the images
# Similar results, not significantlly better 
config_5 = [
    {"name": "baseline_7_classes_dense"},
    {"name": "A3B_relu_k5_improved_7_classes_dense", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5}, 
    {"name": "B2B_deep_bn_7_classes_dense", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True}, 
    {"name": "A4B_relu_k7_7_classes_dense", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128],"dense_units":[128,64]},
]

# sixth batch, using previous. models but on grayscale image
# yet again, not much change
config_6 = [
    {"name": "baseline_7_classes_dense"},
    {"name": "A3B_relu_k5_improved_7_classes_dense", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5,"channels":1}, 
    {"name": "B2B_deep_bn_7_classes_dense", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"channels":1}, 
    {"name": "A4B_relu_k7_7_classes_dense", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128],"dense_units":[128,64],"channels":1},
]

# for batch 7, I will do the training using the previous model on grayscale and 3 channel images
# I Messed up and overwrote the grayscale models by the color models. However, the results were extraordinary.
# 🔍 B2B_deep_bn_7_classes_dense — Evaluation on Test Set:loss: 1.0621 accuracy: 0.7415
# A3B_relu_k5_improved_7_classes_dense — Evaluation on Test Set:loss: 1.0519 accuracy: 0.7343

config_7 = [
    # 3 channels
    {"name": "baseline_7_classes_aug","channels":3,"aug_data":True},
    {"name": "A3B_relu_k5_improved_7_classes_dense", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5,"channels":3,"aug_data":True}, 
    {"name": "B2B_deep_bn_7_classes_dense", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"channels":3,"aug_data":True}, 
    {"name": "A4B_relu_k7_7_classes_dense", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128],"dense_units":[128,64],"channels":3,"aug_data":True},
]

# Batch 8, I'll do the same test but with gray scale to check for differences.
# didn't go better, had sligly less accurate models. Gray scale is not good. 
# Also, due to the size of the augmented data, training takes way way way longer.
# So I have decided to only continue with the top 2 models so far, that being the B2B_deep_bn_7_classes_dense and A3B_relu_k5_improved_7_classes_dense

config_8 = [
    # grayscale channels
    {"name": "baseline_7_classes_aug_gray","channels":1,"aug_data":True},
    {"name": "A3B_relu_k5_improved_7_classes_dense_aug_gray", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5,"channels":1,"aug_data":True}, 
    {"name": "B2B_deep_bn_7_classes_dense_aug_gray", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"channels":1,"aug_data":True}, 
    {"name": "A4B_relu_k7_7_classes_dense_aug_gray", "activation": "relu", "kernel_size": 7, "conv_filters": [32, 64, 128],"dense_units":[128,64],"channels":1,"aug_data":True},
]
# batch 9, I used the functional model api to make it to allow skip connections,
# this should, in theory, make the large CNN perform better.
# updated batch_size to 512 for faster training
# FINALLY, I broke through the ice I had set in my head as the minimum, that being 80%+ accuracy on the test data set.
# That is also where I am doing to stop. 
# The grand winners for the classification are:
#  B2B_deep_bn_7_classes_dense_aug_batch_norm_skip and  A3B_relu_k5_improved_7_classes_dense_aug_skip
config_9=[
    #skip true , batch norm true, batch norm dense true
    {"name": "A3B_relu_k5_improved_7_classes_dense_aug_batch_norm_skip", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128],"batch_norm": True,"batch_norm_dense": True, "dropout_rate": 0.5,"aug_data":True,"use_skip": True},
    
    {"name": "B2B_deep_bn_7_classes_dense_aug_batch_norm_skip", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"batch_norm_dense": True,"aug_data":True,"use_skip":True},
    
    # skip true, batch norm false, batch norm dense false
    {"name": "A3B_relu_k5_improved_7_classes_dense_aug_skip", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5,"aug_data":True,"use_skip": True},
    
    {"name": "B2B_deep_bn_7_classes_dense_aug_skip", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5,"aug_data":True,"use_skip":True},
]


# Regression time

# batch one 1, I'll just train the best model I found so far and go from there
# so bad, so many mistakes i made, so many things i forgot. It just failed miserably, MSE is massive.
# Will train all the previous models and see where to go from there
# Who would've thought, most are not just awful, but outright sad. Only one got a decent results and that is

config_10=[
    {"name": "baseline"},
    {"name": "baseline_non_normalized", "normalize": False},
    {"name": "baseline_dense"},
    {"name": "baseline_aug", "aug_data": True},
    {"name": "baseline_aug_gray", "channels": 1, "aug_data": True},

    {"name": "A1_relu_k3"},
    {"name": "A3_relu_k5", "kernel_size": 5},

    {"name": "B2_deep", "conv_filters": [32, 64, 128, 256, 512]},

    {"name": "J1_dense_512", "dense_units": [512]},
    {"name": "J2_dense_256_128", "dense_units": [256, 128]},
    {"name": "E1_dense1"},
    {"name": "E2_dense3", "dense_units": [256, 128, 64]},

    {"name": "F1_gray", "channels": 1},
    {"name": "F2_gray_k5", "channels": 1, "kernel_size": 5},
    {"name": "F3_gray_deep", "channels": 1, "conv_filters": [32, 64, 128, 256]},
    {"name": "F4_gray_dense2", "channels": 1, "dense_units": [256, 128]},

    {"name": "H1_dropout_0.3", "dropout_rate": 0.3},
    {"name": "H2_dropout_0.7", "dropout_rate": 0.7},

    {"name": "B3_batch_128", "batch_size": 128},
    {"name": "B4_batch_64", "batch_size": 64},

    {"name": "E1_epochs_60", "epochs": 60},
    {"name": "E2_epochs_180", "epochs": 180},
    {"name": "E3_epochs_240", "epochs": 240},
    {"name": "E4_epochs_300", "epochs": 300},

    {"name": "K1_k7_deep", "kernel_size": 7, "conv_filters": [64, 128, 256]},

    {"name": "B2B_deep_bn", "conv_filters": [32, 64, 128, 256, 512], "batch_norm": True},
    {"name": "K1B_k7_deep_bn", "kernel_size": 7, "conv_filters": [64, 128, 256, 512],
     "dropout_rate": 0.4, "batch_norm": True},
    {"name": "E4B_epochs_300", "epochs": 300, "lr_schedule": "reduce_on_plateau"},
    {"name": "J2B_dense_256_128", "dense_units": [256, 128], "batch_norm_dense": True},
    {"name": "A3B_relu_k5_improved", "kernel_size": 5, "conv_filters": [32, 64, 128, 256]},
    {"name": "I1B_deep_relu_bn", "conv_filters": [64, 128, 256, 512], "dropout_rate": 0.4},
    {"name": "I2B_mid_relu"},
    {"name": "A4B_relu_k7", "kernel_size": 7},
    {"name": "A2B_relu_k3_deeper", "conv_filters": [32, 64, 128, 256]},
    {"name": "B1B_deep_dense", "dense_units": [256, 128]},

    {"name": "A3B_relu_k5_improved_reg", "kernel_size": 5, "conv_filters": [32, 64, 128, 256]},
    {"name": "B2B_deep_bn_reg", "conv_filters": [32, 64, 128, 256, 512], "batch_norm": True},
    {"name": "A4B_relu_k7_reg", "kernel_size": 7},
    {"name": "A3B_relu_k5_improved_reg_non_norm", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "normalize": False},
    {"name": "B2B_deep_bn_reg_non_norm", "conv_filters": [32, 64, 128, 256, 512],
     "batch_norm": True, "normalize": False},
    {"name": "A4B_relu_k7_reg_non_norm", "kernel_size": 7, "normalize": False},
    {"name": "A3B_relu_k5_improved_reg_dense", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128]},
    {"name": "B2B_deep_bn_reg_dense", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "batch_norm": True},
    {"name": "A4B_relu_k7_reg_dense", "kernel_size": 7, "dense_units": [128, 64]},
    {"name": "A3B_relu_k5_improved_reg_dense_gray", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128], "channels": 1},
    {"name": "B2B_deep_bn_reg_dense_gray", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "batch_norm": True, "channels": 1},
    {"name": "A4B_relu_k7_reg_dense_gray", "kernel_size": 7, "channels": 1},
    {"name": "A3B_relu_k5_improved_reg_aug", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128], "aug_data": True},
    {"name": "B2B_deep_bn_reg_aug", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "batch_norm": True, "aug_data": True},
    {"name": "A4B_relu_k7_reg_aug", "kernel_size": 7, "aug_data": True},
    {"name": "A3B_relu_k5_improved_reg_aug_gray", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128], "channels": 1, "aug_data": True},
    {"name": "B2B_deep_bn_reg_aug_gray", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "batch_norm": True, "channels": 1, "aug_data": True},
    {"name": "A4B_relu_k7_reg_aug_gray", "kernel_size": 7, "channels": 1, "aug_data": True},
    {"name": "A3B_relu_k5_improved_reg_aug_bn_skip", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128], "batch_norm": True, "batch_norm_dense": True,
     "aug_data": True, "use_skip": True},
    {"name": "B2B_deep_bn_reg_aug_bn_skip", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "batch_norm": True, "batch_norm_dense": True,
     "aug_data": True, "use_skip": True},
    {"name": "A3B_relu_k5_improved_reg_aug_skip", "kernel_size": 5, "conv_filters": [32, 64, 128, 256],
     "dense_units": [256, 128], "aug_data": True, "use_skip": True},
    {"name": "B2B_deep_bn_reg_aug_skip", "conv_filters": [32, 64, 128, 256, 512],
     "dense_units": [512, 256, 128], "aug_data": True, "use_skip": True},
]

# I have no idea why but B2B_deep_bn_reg_dense_gray got an outlier of a result, 51 MSE. I will check this one in depth with more epochs and see what's happening
# Ok so, i found out somehting interesting, my validation loss goes very high in the first 30 - ish epochs and then 
# comes down to a decent value. The early stopping was causing the models to be pretty much random,
# 
config_11 = [
    {"name": "B2B_deep_bn_reg_dense_gray", "conv_filters": [32, 64, 128, 256, 512],"dense_units": [512, 256, 128], "batch_norm": True, "channels": 1},
    {"name": "B2B_deep_bn","conv_filters": [32, 64, 128, 256, 512],"batch_norm": True,}

]
# Trained the best again with delayed early stopping and 120 epochs.
config_12 = [
    {"name": "B2B_deep_bn_reg_dense_gray", "conv_filters": [32, 64, 128, 256, 512],"dense_units": [512, 256, 128], "batch_norm": True, "channels": 1},
    {"name": "B2B_deep_bn_reg", "conv_filters": [32, 64, 128, 256, 512], "batch_norm": True},
    
    {"name": "A3B_relu_k5_reg_dense_aug_bn_skip", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128],"batch_norm": True,"batch_norm_dense": True, "dropout_rate": 0.5,"aug_data":True,"use_skip": True},
    
    {"name": "B2B_deep_bn_reg_dense_aug_bn_skip", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"batch_norm_dense": True,"aug_data":True,"use_skip":True},
    
    {"name": "A3B_relu_k5_reg_dense_aug_skip", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128], "dropout_rate": 0.5,"aug_data":True,"use_skip": True},
    
    {"name": "B2B_deep_reg_dense_aug_skip", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5,"aug_data":True,"use_skip":True},
]

# not half bad. some decent results, I would say, swtiched to mae for validation because it makes a bit more sense to me.
# How far off are we from the real age? That's pretty much it. And I got a 7.17 and a 7.99. Not good, but not bad
# definitely way better than some other ones.
config_13 = [
{"name": "B2B_deep_bn_reg_aug_bn_skip", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "batch_norm_dense": True, "aug_data": True, "use_skip": True},
{"name": "B2B_deep_bn_reg_aug", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},
{"name": "B2B_deep_bn_reg_aug_skip", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "aug_data": True, "use_skip": True},
{"name": "A3B_relu_k5_improved_reg_aug", "kernel_size": 5, "conv_filters": [32,64,128,256], "dense_units": [256,128], "aug_data": True},
{"name": "B2B_deep_bn_reg_aug_gray", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "channels": 1, "aug_data": True},

# added l2 reglr
{"name": "B2B_deep_bn_reg_dense_gray", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "channels": 1,"l2_reg":1e-4},
{"name": "B2B_deep_bn_reg", "conv_filters": [32,64,128,256,512], "batch_norm": True,"l2_reg":1e-4},
{"name": "B2B_deep_bn", "conv_filters": [32,64,128,256,512], "batch_norm": True, "l2_reg":1e-4},


]

# I'll take the top three from the last training and just train them for longer

# B2B_deep_bn_reg_aug	103.85	7.17	
# B2B_deep_bn_reg_aug_gray	136.48	7.99
# B2B_deep_bn_reg_dense_gray	232.58	10.67	


# Marginally better results  on B2B_deep_bn_reg_aug_gray    mse: 118.6801 mae: 7.4108
# same for B2B_deep_bn_reg_dense_gray mse: 201.5796 mae: 9.9408


config_14 = [
{"name": "B2B_deep_bn_reg_aug","epochs":500, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},
{"name": "B2B_deep_bn_reg_aug_gray","epochs":500, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "channels": 1, "aug_data": True},
{"name": "B2B_deep_bn_reg_dense_gray","epochs":500, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "channels": 1,"l2_reg":1e-4},

# for funsies:
{"name": "B2B_deep_bn_reg_aug","epochs":500,"activation":"linear", "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True}
]
# one last try, adding skip connections and l2 to the best model
# CNN_tuned_small_4block_bn_dropout_gap mse: 316.8465 mae: 13.1326
# B2B_deep_bn_reg_aug_skip_l2 mse: 152.3895 mae: 8.9242
config_15 = [
{"name": "CNN_tuned_small_4block_bn_dropout_gap", "output_activation": "relu", "conv_filters": [32, 64, 128, 256], "dense_units": [128], "batch_norm": True, "dropout_rate": 0.5, "aug_data": True, "l2_reg": 1e-4, "batch_size": 1024},
{"name": "B2B_deep_bn_reg_aug_skip_l2","output_activation":"linear","batch_size": 1024,"epochs":500, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True,"l2_reg":1e-4,"use_skip":True},
]


config_16 = [
#     loss: 387.7751 mae: 14.8765
 
{"name": "CNN_tuned_small_4block_bn_dropout_gap","epochs":1000, "output_activation": "relu", "conv_filters": [32, 64, 128, 256], "dense_units": [128], "batch_norm": True, "dropout_rate": 0.5, "aug_data": False, "l2_reg": 1e-4, "batch_size": 256},

]
# Absolutely insane performance pulled off here. 5.5 MAE on test set
#  Will let it train for even longer by disabling the early stopper maybe
# I believe it can be even better 
config_17=[
# B2B_deep_bn_reg_aug  mse: 63.5034 mae: 5.5665
{"name": "B2B_deep_bn_reg_aug","epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},
{"name": "B2B_deep_bn_reg_aug_linear", "activation": "linear","epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},

]

# now with these two nice models, I'll try again them for the classification
# B2B_deep_bn_reg_aug_classification_7_class — Evaluation on Test Set: loss: 0.8003 accuracy: 0.6729

# B2B_deep_bn_reg_aug_classification_13_class — Evaluation on Test Set: loss: 0.7616 accuracy: 0.6855

confi_18=[
{"name": "B2B_deep_bn_reg_aug_classification_7_class","task":"classification","num_classes":7,"epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},
{"name": "B2B_deep_bn_reg_aug_classification_13_class","task":"classification","num_classes":13,"epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True},
]


In [17]:
config_count = 19

# To try to train these again to try to get the same accuracy again.
#   {"name": "A3B_relu_k5_improved_7_classes_dense_aug_batch_norm_skip", "activation": "relu","kernel_size": 5, "conv_filters": [32, 64, 128, 256],"dense_units":[256,128],"batch_norm": True,"batch_norm_dense": True, "dropout_rate": 0.5,"aug_data":True,"use_skip": True}, 
# {"name": "B2B_deep_bn_7_classes_dense_aug_batch_norm_skip", "conv_filters": [32, 64, 128, 256, 512],"dense_units":[512,256,128], "activation": "relu", "dropout_rate": 0.5, "batch_norm": True,"batch_norm_dense": True,"aug_data":True,"use_skip":True},
    
configs = [
    {"name": "B2B_deep_bn_reg_aug_autoencoder","epochs":500, "l2_reg": 1e-4,"batch_size":256,"batch_norm_dense":True, "conv_filters": [32,64,128,256,512], "dense_units": [512,256,128], "batch_norm": True, "aug_data": True,"task":"regression","resize":(192,192)},
    
]


default_args = {
    "batch_size": 512,
    "epochs": 200,
    "channels": 3,
    "dropout_rate": 0.5,
    "task": "regression",
    "num_classes": 7,
    "conv_filters": [32, 64, 128],
    "kernel_size": 3,
    "activation": "relu",
    "dense_units": [128],
    "output_activation": "softmax",
    "batch_norm" : False,
    "batch_norm_dense": False,
    "normalize" : True,
    "aug_data": False,
    "use_skip": False,
    "l2_reg":0
    
}


ratio=1
# Build all models and store in DataFrame
model_list = []
for cfg in configs:
    cfg_full = {**default_args, **cfg}
    cfg_full["loss"]='categorical_crossentropy' if cfg_full["task"] =="classification" else "mse"
    cfg_full["metrics"]=['accuracy'] if cfg_full["task"] =="classification" else ["mse","mae"]
    
    
    dict_for_data= aug_data if cfg_full["aug_data"] else data
    label_kind = "labels" if cfg_full["task"] =="classification" else "labels_regr"
    train_dataset = load_images_from_paths(dict_for_data["train"]["filenames"],
                                           dict_for_data["train"][label_kind], 
                                           channels=cfg_full["channels"],
                                           ratio=ratio,
                                           batch_size=cfg_full['batch_size'],
                                           class_count=cfg_full['num_classes'],
                                           task=cfg_full['task'])
    
    val_dataset = load_images_from_paths(data["val"]["filenames"], data["val"][label_kind],
                                         channels=cfg_full["channels"],
                                         ratio=ratio,
                                         batch_size=cfg_full['batch_size'],
                                         class_count=cfg_full['num_classes'],
                                         task=cfg_full['task'])

    test_dataset = load_images_from_paths(data["test"]["filenames"], data["test"][label_kind],
                                         channels=cfg_full["channels"],
                                         ratio=ratio,
                                         batch_size=cfg_full['batch_size'],
                                         class_count=cfg_full['num_classes'],
                                         task=cfg_full['task'])
    
    # if skip, make it functional 
    model = build_cnn_model(
            channels=cfg_full["channels"],
            dropout_rate=cfg_full["dropout_rate"],
            task=cfg_full["task"],
            num_classes=cfg_full["num_classes"],
            conv_filters=cfg_full["conv_filters"],
            kernel_size=cfg_full["kernel_size"],
            activation=cfg_full["activation"],
            dense_units=cfg_full["dense_units"],
            output_activation=cfg_full["output_activation"],
            batch_norm=cfg_full["batch_norm"],
            batch_norm_dense=cfg_full["batch_norm_dense"],
            use_skip=cfg_full["use_skip"],
    )

    
    optimizer = Adam(learning_rate=0.00001)
    model.compile(loss=cfg_full["loss"], optimizer=optimizer, metrics=cfg_full["metrics"])
    cfg_full["model"] = model
    cfg_full["train_data"]=train_dataset
    cfg_full["val_data"]=val_dataset
    cfg_full["test_data"]=test_dataset
    
    model_list.append(cfg_full)




In [18]:
model_list[0]['model'].summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 190, 190, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 190, 190, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_10            │ (None, 95, 95, 32)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 93, 93, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 93, 93, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_11            │ (None, 46, 46, 64)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 44, 44, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 44, 44, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_12            │ (None, 22, 22, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 20, 20, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_13            │ (None, 10, 10, 256)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_14            │ (None, 4, 4, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             

 Total params: 2,003,137 (7.64 MB)

 Trainable params: 1,999,361 (7.63 MB)

 Non-trainable params: 3,776 (14.75 KB)

In [ ]:
for config in model_list:
    print(f"\nModel: {config['name']}")
    
    # Create iterators for the datasets
    train_iter = iter(config["train_data"])
    val_iter = iter(config["val_data"])
    test_iter = iter(config["test_data"])
    
    # Print header
    header = "{:<5}{:<25}{:<25}{:<25}".format("Idx", "Train Label", "Val Label", "Test Label")
    print(header)
    print("-" * len(header))
    
    # Iterate for the first 10 batches
    for i in range(2):
        train_sample = next(train_iter, None)
        val_sample = next(val_iter, None)
        test_sample = next(test_iter, None)
        
        # Extract labels; assume each sample is (data, label)
        train_label = train_sample[1] if train_sample is not None else "None"
        val_label = val_sample[1] if val_sample is not None else "None"
        test_label = test_sample[1] if test_sample is not None else "None"
        
        # Convert tensors to lists for readable output (if needed)
        if hasattr(train_label, "numpy"):
            train_label = train_label.numpy().tolist()
        if hasattr(val_label, "numpy"):
            val_label = val_label.numpy().tolist()
        if hasattr(test_label, "numpy"):
            test_label = test_label.numpy().tolist()
        
        # Print the formatted row
        row = "{:<5}{:<25}{:<25}{:<25}".format(i, str(train_label), str(val_label), str(test_label))
        print(row)
    
    print("-" * len(header))


In [ ]:
model_list

# Model Fitting

In [ ]:
os.makedirs(f"regression_saved_models_config_{config_count}", exist_ok=True)
os.makedirs(f"classification_saved_models_config_{config_count}", exist_ok=True)


for entry in model_list:
    model_name = entry["name"]
    
    # Setup model checkpoint callback
    checkpoint_path = f"{entry['task']}_saved_models_config_{config_count}/{model_name}_best_model.keras"
    mse_checkpoint = ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_accuracy' if entry["task"]=='classification' else 'val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=0
    )
    entry['best_model_path'] = checkpoint_path

    callbacks = [mse_checkpoint]

    if entry["task"]!='classification':
        mae_checkpoint = ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            verbose=0
        )
        callbacks.append(mae_checkpoint)

    # useless or I'm dumb and don't know how to use it properly
    if "lr_schedule" in entry:
        if entry["lr_schedule"] == "reduce_on_plateau":
            lr_callback = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                verbose=1
            )
            callbacks.append(lr_callback)

    early_stop_loss = DelayedEarlyStopping(
        monitor='val_loss',
        patience=10,
        verbose=0,
        restore_best_weights=False,
        delay=60  # begin checks after 60 epochs
    )
    
    if entry["task"]=='classification':
        early_stop_acc = DelayedEarlyStopping(
            monitor='val_accuracy',
            patience=7,
            verbose=0,
            restore_best_weights=True,
            delay=60
        )
        callbacks.append(early_stop_acc)

    callbacks.append(early_stop_loss)
    callbacks.append(LiveLossPlot(model_name=model_name))

    # Train model
    history = entry['model'].fit(
        entry['train_data'],
        validation_data=entry['val_data'],
        epochs=entry['epochs'],
        batch_size=entry['batch_size'],
        verbose=1,
        callbacks=callbacks
    )

    # Save training history in entry
    entry['history'] = history.history
    entry['best_model_path'] = checkpoint_path
    df_model_summary = pd.DataFrame([
    {k: v for k, v in entry.items() if k not in ['model', 'train_data', 'val_data','test_data']} for entry in model_list])
    df_model_summary.to_csv(f"regr_model_summary_config_{config_count}.csv", index=False)

    


    

# Checking Model Performance

In [10]:

# Create subplots: one row per model, two columns (loss and accuracy)

fig, axes = plt.subplots(nrows=len(model_list), ncols=2, figsize=(15, 5 * len(model_list)))

# Ensure axes is always 2D
if len(model_list) == 1:
    axes = np.array([axes])

# Loop through each trained model entry
for i, entry in enumerate(model_list):
    try:
        model_name = entry["name"]
        history = entry["history"]
        epochs = list(range(1, len(history["loss"]) + 1))

        # Loss Plot
        ax_loss = axes[i, 0]
        ax_loss.plot(epochs, history["loss"], label='Train Loss', marker='o')
        ax_loss.plot(epochs, history["val_loss"], label='Val Loss', marker='o')
        ax_loss.set_title(f"{model_name} - Loss")
        ax_loss.set_xlabel("Epoch")
        ax_loss.set_ylabel("Loss")
        ax_loss.legend()

        # Accuracy Plot
        ax_acc = axes[i, 1]
        ax_acc.plot(epochs, history["accuracy"], label='Train Accuracy', marker='o')
        ax_acc.plot(epochs, history["val_accuracy"], label='Val Accuracy', marker='o')
        ax_acc.set_title(f"{model_name} - Accuracy")
        ax_acc.set_xlabel("Epoch")
        ax_acc.set_ylabel("Accuracy")
        ax_acc.legend()
    except Exception:
        continue

plt.tight_layout()
plt.show()

NameError: name 'model_list' is not defined

In [ ]:
def get_preds_and_labels(model, dataset):
    y_pred = []
    y_true = []
    for batch in dataset:
        images, labels = batch
        probs = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(probs, axis=-1))
        y_true.extend(labels.numpy())
    return np.array(y_pred), np.array(y_true)


In [ ]:
model_list

In [11]:
import ast
ratio=1
model_list_loaded=pd.read_csv(f"regression_model_summary_config_16.csv")
model_list = []
for _, row in model_list_loaded.iterrows():
    if pd.notna(row["history"]):
        history_dict = ast.literal_eval(row["history"])
        print(row["aug_data"])
        model_list.append({
            "name": row["name"],
            "history": history_dict,
            "aug_data": row["aug_data"],
            "channels": row["channels"],
            "batch_size": row["batch_size"],
            "num_classes": row["num_classes"],
            "normalize": row["normalize"],
            "task": row["task"]
        })


FileNotFoundError: [Errno 2] No such file or directory: 'regression_model_summary_config_16.csv'

In [ ]:

# Create subplots: one row per model, two columns (loss and accuracy)

fig, axes = plt.subplots(nrows=len(model_list), ncols=2, figsize=(15, 5 * len(model_list)))

# Ensure axes is always 2D
if len(model_list) == 1:
    axes = np.array([axes])

# Loop through each trained model entry
for i, entry in enumerate(model_list):
    try:
        model_name = entry["name"]
        history = entry["history"]
        epochs = list(range(1, len(history["loss"]) + 1))

        # Loss Plot
        ax_loss = axes[i, 0]
        ax_loss.plot(epochs, history["loss"], label='Train Loss', marker='o')
        ax_loss.plot(epochs, history["val_loss"], label='Val Loss', marker='o')
        ax_loss.set_title(f"{model_name} - Loss")
        ax_loss.set_xlabel("Epoch")
        ax_loss.set_ylabel("Loss")
        ax_loss.legend()

        # Accuracy Plot
        ax_acc = axes[i, 1]
        ax_acc.plot(epochs, history["accuracy"], label='Train Accuracy', marker='o')
        ax_acc.plot(epochs, history["val_accuracy"], label='Val Accuracy', marker='o')
        ax_acc.set_title(f"{model_name} - Accuracy")
        ax_acc.set_xlabel("Epoch")
        ax_acc.set_ylabel("Accuracy")
        ax_acc.legend()
    except Exception:
        continue

plt.tight_layout()
plt.show()

In [ ]:


for entry in model_list:
    # try:
        # print(entry["aug_data"])
        dict_for_data= aug_data if entry["aug_data"] else data
        label_kind = "labels" if entry["task"] =="classification" else "labels_regr"


        test_dataset = load_images_from_paths(data["test"]["filenames"], data["test"][label_kind],
                                         channels=entry["channels"],
                                         ratio=ratio,
                                         batch_size=entry['batch_size'],
                                         class_count=entry['num_classes'],
                                         normalize=entry['normalize'],
                                         task=entry["task"])


        entry["test_data"]=test_dataset

    # except Exception:
        # print("bad")
        # continue
    


In [ ]:
%whos 

In [ ]:

config_count=9
# Loop through each trained model entry
for i, entry in enumerate(model_list):
    
        model_name = entry["name"]
        best_model_path = f"saved_models_config_{config_count}/{model_name}_best_model.keras"
        
        test_dataset = entry['test_data']
        # Load the best saved model
        
        model = load_model(best_model_path)
        

        # Evaluate on test set
        model_score = model.evaluate(test_dataset, verbose=1)
        model_labels = model.metrics_names

        print(f"🔍 {model_name} — Evaluation on Test Set:")
        for label, score in zip(model_labels, model_score):
            print(f"   {label}: {round(score, 4)}")
        print("-" * 40)
        y_pred = model.predict(test_dataset)
        y_true = np.concatenate([y for x, y in test_dataset], axis=0)

        # Print predicted vs true values
        print("📋 Predictions vs Ground Truth:")
        # for pred, true in zip(y_pred.flatten(), y_true.flatten()):
            # print(f"   Predicted: {round(pred, 2)} — Actual: {int(true)}")
        pred_classes = np.argmax(y_pred, axis=1)
        true_classes = np.argmax(y_true, axis=1)

        for pred, true in zip(pred_classes, true_classes):
            print(f"   Predicted: {pred} — Actual: {true}")


    

### Plotting Confusion Matrix

In [ ]:
# Defining a function to plot the confusion matrix in a grid for easier visualization.

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', export_as='confusion_matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True labels', fontsize=14)
    plt.xlabel('Predicted labels', fontsize=14)

    # Exporting plot image in PNG format.
    plt.savefig(f'./output/{export_as}.png', bbox_inches='tight')

In [ ]:

# Define class labels
cm_plot_labels = ['1-2', '3-5', '6-8', '9-12', '13-17', '18-24', '25-34', 
                  '35-44', '45-54', '55-64', '65-74', '75-84', '85+']

# Loop through all models
for i, entry in enumerate(model_list):
    model_name = entry["name"]
    
    
    # best_model_path = entry["best_model_path"]
    best_model_path = f"saved_models_config_{config_count}/{model_name}_best_model.keras"
    test_dataset = entry['test_data']

    print(f"🔍 Evaluating Confusion Matrix for: {model_name}")

    # Load the best saved model
    model = load_model(best_model_path)

    # Predict class probabilities
    model_pred_probs = model.predict(test_dataset)
    model_pred = model_pred_probs.argmax(axis=-1)
    y_pred, y_true = get_preds_and_labels(model, test_dataset)

    # y_true = dict_for_data["test"]["labels_regr"]

    # Compute confusion matrix
    if entry["task"]=="classification":
        if len(y_true.shape) > 1 and y_true.shape[1] > 1:  # One-hot check
            y_true = np.argmax(y_true, axis=-1)

        conf_mat = confusion_matrix(y_true, model_pred)

        # Plot confusion matrix
        plt.figure(figsize=(16, 8))
        plot_confusion_matrix(
            conf_mat,
            cm_plot_labels,
            normalize=True,
            title=f"Confusion Matrix - {model_name}",
            export_as=None  # Change to f"{model_name}_confmat" to save
        )
        plt.show()

    print("Sample predictions vs actuals:")
    for pred, true in zip(y_pred[:10], y_true[:10]):
        print(f"Pred: {pred} — True: {true}")



In [ ]:
y_true
